# Data mining - summarization

In [1]:
import pandas as pd
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

import nltk
# nltk.download('punkt')

# https://github.com/miso-belica/sumy

In [11]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/zam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# " ".join(l1)

# paragraph p, sentence s, word w

def paragraphToPlain(l1):
    text = ""
    for p in l1:
        par = ""
        for s in p:
            se = ""
            for w in s:
                se += w + " "
            par += se+" "
        text += par
    return text

In [3]:
jsonObj = pd.read_json("/media/data1/_S2/indosum/train.01.100.jsonl", lines=True)

In [4]:
jsonObj

,category,gold_labels,id,paragraphs,source,source_url,summary
0,tajuk utama,"[[False, True], [True, True], [False, False, F...",1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dokter, R...",cnn indonesia,https://www.cnnindonesia.com/hiburan/201708041...,"[[Dokter, Lula, Kamal, yang, merupakan, selebr..."
1,teknologi,"[[False, False, False, False], [False, True, T...",1509072914-dua-smartphone-zenfone-baru-tawarka...,"[[[Selfie, ialah, salah, satu, tema, terpanas,...",dailysocial.id,https://dailysocial.id/post/dua-smartphone-zen...,"[[Asus, memperkenalkan, , ZenFone, generasi, ..."
2,hiburan,"[[True], [True], [False, False], [False], [Fal...",1510613677-songsong-visit-2020-bengkulu-perkua...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dinas, Pa...",cnn indonesia,https://www.cnnindonesia.com/gaya-hidup/201711...,"[[Dinas, Pariwisata, Provinsi, Bengkulu, kemba..."
3,tajuk utama,"[[True, True], [False, False, False], [True], ...",1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,"[[[Merdeka.com, -, Indonesia, Corruption, Watc...",merdeka,https://www.merdeka.com/peristiwa/icw-merasa-a...,"[[Indonesia, Corruption, Watch, (, ICW, ), mem..."
4,tajuk utama,"[[False, True], [True, True, True], [False], [...",1503039338-pembagian-sepeda-usai-upacara-penur...,"[[[Merdeka.com, -, Presiden, Joko, Widodo, (, ...",merdeka,https://www.merdeka.com/peristiwa/usai-upacara...,"[[Jokowi, memimpin, upacara, penurunan, bender..."
...,...,...,...,...,...,...,...
95,tajuk utama,"[[False, False], [False, False], [False], [Fal...",1484474411-reklamasi-jakarta-untuk-siapa,"[[[Jakarta, ,, CNN, Indonesia, -, -, Banjir, y...",cnn indonesia,http://m.cnnindonesia.com/nasional/20170110202...,"[[Permasalahan, tenggelamnya, daratan, di, Jak..."
96,showbiz,"[[True, True], [True, True], [True, False], [F...",1500161400-selena-gomez-akui-punya-fetish-di-l...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Selena, G...",cnn indonesia,https://www.cnnindonesia.com/hiburan/201707131...,"[[Selena, Gomez, baru, mengaku, bahwa, dia, me..."
97,teknologi,"[[False, False], [False], [False, False], [Tru...",1499749200-angin-meningkatkan-peran-perempuan-...,"[[[Permasalahan, masih, minimnya, jumlah, pere...",dailysocial.id,https://dailysocial.id/post/rencana-angin-meni...,"[[Angel, Investment, Network, Indonesia, (, AN..."
98,olahraga,"[[False, True], [False, False], [False, False]...",1505764276-thailand-raih-juara-piala-aff-u-18-...,"[[[Tim, nasional, (, timnas, ), Thailand, berh...",goal indonesia,http://www.goal.com/id-ID/news/1387/nasional/2...,"[[Thailand, menaklukkan, Malaysia, dengan, sko..."


In [5]:
jsonObj.describe()

,category,gold_labels,id,paragraphs,source,source_url,summary
count,100,100,100,100,100,100,100
unique,5,100,100,100,10,100,100
top,tajuk utama,"[[False, True], [True, True], [False, False, F...",1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dokter, R...",cnn indonesia,https://www.cnnindonesia.com/hiburan/201708041...,"[[Dokter, Lula, Kamal, yang, merupakan, selebr..."
freq,42,1,1,1,43,1,1


In [6]:
jsonObj['paragraphs'][0]

[[['Jakarta',
   ',',
   'CNN',
   'Indonesia',
   '-',
   '-',
   'Dokter',
   'Ryan',
   'Thamrin',
   ',',
   'yang',
   'terkenal',
   'lewat',
   'acara',
   'Dokter',
   'Oz',
   'Indonesia',
   ',',
   'meninggal',
   'dunia',
   'pada',
   'Jumat',
   '(',
   '4',
   '/',
   '8',
   ')',
   'dini',
   'hari',
   '.'],
  ['Dokter',
   'Lula',
   'Kamal',
   'yang',
   'merupakan',
   'selebriti',
   'sekaligus',
   'rekan',
   'kerja',
   'Ryan',
   'menyebut',
   'kawannya',
   'itu',
   'sudah',
   'sakit',
   'sejak',
   'setahun',
   'yang',
   'lalu',
   '.']],
 [['Lula',
   'menuturkan',
   ',',
   'sakit',
   'itu',
   'membuat',
   'Ryan',
   'mesti',
   'vakum',
   'dari',
   'semua',
   'kegiatannya',
   ',',
   'termasuk',
   'menjadi',
   'pembawa',
   'acara',
   'Dokter',
   'Oz',
   'Indonesia',
   '.'],
  ['Kondisi',
   'itu',
   'membuat',
   'Ryan',
   'harus',
   'kembali',
   'ke',
   'kampung',
   'halamannya',
   'di',
   'Pekanbaru',
   ',',
   'Riau',
   

In [7]:
p1 = eval(str(jsonObj['paragraphs'][0]))

In [8]:
text = paragraphToPlain(p1)
text

'Jakarta , CNN Indonesia - - Dokter Ryan Thamrin , yang terkenal lewat acara Dokter Oz Indonesia , meninggal dunia pada Jumat ( 4 / 8 ) dini hari .  Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu .  Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia .  Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat .  " Setahu saya dia orangnya sehat , tapi tahun lalu saya dengar dia sakit .  ( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah .  Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) .  Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh .  Dia juga tak tahu penyakit apa yang did

In [14]:
# LANGUAGE = "english"
# SENTENCES_COUNT = 10

# sample api = https://github.com/miso-belica/sumy
def summarize(string, LANGUAGE = "english", SENTENCES_COUNT = 10):
    #url = "https://en.wikipedia.org/wiki/Automatic_summarization"
    #parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    # or for plain text files
    # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
    parser = PlaintextParser.from_string(string, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    sums = []
    
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
        sums.append(str(sentence))
    
    return string, sums


In [15]:
summarize(text, LANGUAGE = "english", SENTENCES_COUNT = 10)

Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia .
Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat .  "
( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah .
Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) .
Dia juga tak tahu penyakit apa yang diderita Ryan .  "
Enggak tahu berat sekali apa bagaimana , " tutur Ryan .
Walau sudah setahun menderita sakit , Lula tak mengetahui apa penyebab pasti kematian Dr Oz Indonesia itu .
Meski demikian , ia mendengar beberapa kabar yang menyebut bahwa penyebab Ryan meninggal adalah karena jatuh di kamar mandi .
Ryan menempuh Pendidikan Dokter pada tahun 2002 di Fakultas Kedokteran Universitas Gadjah Mada .
Dia kemudian melanjutkan pendidikan Klinis Kesehatan Reproduksi dan Penyakit Menular

('Jakarta , CNN Indonesia - - Dokter Ryan Thamrin , yang terkenal lewat acara Dokter Oz Indonesia , meninggal dunia pada Jumat ( 4 / 8 ) dini hari .  Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu .  Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia .  Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat .  " Setahu saya dia orangnya sehat , tapi tahun lalu saya dengar dia sakit .  ( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah .  Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) .  Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh .  Dia juga tak tahu penyakit apa yang di